In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
from pyspark.sql.types import ArrayType
import re
import time

start_time = time.time()
spark = SparkSession.builder.master("spark://sparkmaster:7077").appName("extract_wiki_dict").getOrCreate()
sc = spark.sparkContext

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/05 20:14:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/12/05 20:14:11 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/12/05 20:14:47 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
21/12/05 20:14:47 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:919)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:154)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEnd

In [6]:
# extract text tags from xml
initial_df = spark.read.format('xml').options(rowTag='page').load('./wiki.xml')
df = initial_df.selectExpr("revision.text._VALUE as text")

In [16]:
# clean text
import pyspark.sql.functions as sqlf
df.select(sqlf.regexp_replace('text', r'(<ref.+?/>)|(<ref.+?</ref>)|(<!--.+?-->)|(\s?(\(([^()])*\)))', ''))

DataFrame[regexp_replace(text, (<ref.+?/>)|(<ref.+?</ref>)|(<!--.+?-->)|(\s?(\(([^()])*\))), , 1): string]

In [4]:
#tokenize to sentences
from pyspark.ml.feature import RegexTokenizer
sen_tok = RegexTokenizer(inputCol='text', outputCol='sentences', gaps=False, pattern=r'((\s|^)\'*[A-Z].+?[.!?])(?=\s+\S*[A-Z]|$)')

In [5]:

def tokenize_sentence(sentence):
    results = re.findall(r'[^\s!,.?":;]+', sentence)
    return results

In [ ]:
def clean_sentences(sentence_tuples):
    entries = []
    for sentence_tuple in sentence_tuples:
        original = sentence_tuple[0]
        # print(f'Origin: {original}')
        processed = re.sub(r'(\'\'\'?)|(^\s)|(<.+?>)|(</.+?>)', '', original)
        results = extract_links_to_dict(processed)
        processed = re.sub(r'(\[\[[^]]*\|)|(]])|(\[\[)', '', processed)
        print(f'Clear: {processed}')
        lemmatized = lemmatize_sentence(processed, results)
        print(f'Lemmatized: {lemmatized}')
        entries.append({
            "original": original,
            "processed": processed,
            "lemmatized": lemmatized
        })

    return entries

def parse_text_to_sentences(text):
    text = re.sub(r'(<ref.+?/>)|(<ref.+?</ref>)|(<!--.+?-->)|(\s?(\(([^()])*\)))', '', text)
    sentences = re.findall(r'((\s|^)\'*[A-Z].+?[.!?])(?=\s+\S*[A-Z]|$)', text)
    # sentences = re.findall(r'(\S*[A-Z].+?[.!?])(?=\s+\S*[A-Z]|$)', text)
    # sentences = re.findall(r'(\S*[A-Z].+?(\(.+?\))?[.!?])(?=\s+\S*[A-Z]|$)', text)
    # sentences = re.findall(r'(?![a-z])*', text)
    entries = clean_sentences(sentences)
    return entries

In [6]:

def parse_page(page_string):
    text = parse_text_from_xml(page_string)
    entries = parse_text_to_sentences(text)
    return entries


131.79812622070312


In [ ]:
def parse_text_from_xml(xml_string):
    root = ElementTree.fromstring(xml_string)
    text_element = root.find('revision/text')
    return text_element.text

In [ ]:
def lemmatize_sentence(sentence, link_dict):

    words = tokenize_sentence(sentence)
    processed = ''
    for word in words:
        lemma = simplemma.lemmatize(word, lang_data)
        processed += lemma + ' '
    return processed

In [ ]:
def is_dict_pair_valid(dict_pair):
    # If the starting letter differs, the pair is not valid
    if not dict_pair['base'][0] == dict_pair['form'][0]:
        return False
    # If the number of words differ, the pair is not valid
    if not len(dict_pair['base'].split(' ')) == len(dict_pair['form'].split(' ')):
        return False
    return True

def extract_links_to_dict(sentence):
    dict_results = []
    results = re.findall(r'\[\[[A-Za-z0-9.]+?\|.+?]]', sentence)
    if results:
        for result in results:
            dict_result = {
                'base': re.findall(r'\[\[(.+?)\|', result)[0],
                'form': re.findall(r'\|(.+?)]]', result)[0],
                'postfix': ''
            }
            if is_dict_pair_valid(dict_result):
                dict_results.append(dict_result)
    if not results:
        results = re.findall(r'\[\[[A-Za-z0-9.]+?]][a-z]+?\s', sentence)
        if results:
            for result in results:
                dict_result = {
                    'base': re.findall(r'\[\[(.+?)]]', result)[0],
                    'postfix': re.findall(r'\[\[.+?]](.*)\s', result)[0]
                }
                dict_result['form'] = dict_result['base'] + dict_result['postfix']
                dict_results.append(dict_result)

    if results:
        print(dict_results)
    save_to_terms_dictionary(dict_results)
    return dict_results
